In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

2024-05-27 10:54:50.184006: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-27 10:54:50.228857: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-27 10:54:50.229831: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-27 10:54:51.090962: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
data = pd.read_csv("all-data-processed-3classes.csv")

In [3]:
data

,Sentiment,Sentence,Clean sentences
0,-1,In Finland 's Hobby Hall 's sales decreased by...,finland hobby hall sales decreased 10 internat...
1,0,Panostaja did not disclose the purchase price .,panostaja disclose purchase price
2,-1,Scanfil will execute the temporary lay-offs by...,scanfil execute temporary layoffs midoctober 2...
3,1,Operating profit rose to EUR 13.1 mn from EUR ...,operating profit rose eur 131 mn eur 87 mn cor...
4,0,The company 's share is quoted on NASDAQ OMX H...,company share quoted nasdaq omx helsinki rauta...
...,...,...,...
8632,0,In addition to the presentations held by Presi...,addition presentations held president ceo kai ...
8633,1,"However , the broker gave an `` outperform '' ...",however broker gave outperform recommendation ...
8634,0,Tampere Science Parks is a Finnish company tha...,tampere science parks finnish company owns lea...
8635,0,Aldata noted that its Voice Supply Chain Techn...,aldata noted voice supply chain technology app...


In [4]:
X_train, X_test, y_train, y_test = train_test_split(data["Clean sentences"], data["Sentiment"], test_size=0.1, random_state=42)
X_train.shape , X_test.shape, y_train.shape, y_test.shape

((7773,), (864,), (7773,), (864,))

tokenizing (str to int conversion)

In [5]:
X_train

4655    manager critical politicians failure different...
7959    russia ready participate open tender latteleco...
37      developments partly reflect government higher ...
5761    sampo group become major shareholder nordea ow...
8605    seawind en route finnish port turku stockholm ...
                              ...                        
5734    webcast may followed online company website ww...
5191    according finnish scanfil founder chairman boa...
5390    currency conversions based exchange rates time...
860     thus method cut working costs fasten planning ...
7270    omx helsinki 25 index 092 pct 251867 helsinki ...
Name: Clean sentences, Length: 7773, dtype: object

In [6]:
X_train = X_train.astype(str)

In [7]:
token = Tokenizer()
token.fit_on_texts(X_train)

In [8]:
vocab = len(token.index_word) + 1
print("Vocabulary size={}".format(len(token.word_index)))
print("Number of Documents={}".format(token.document_count))

Vocabulary size=10468
Number of Documents=7773


In [9]:
X_train = token.texts_to_sequences(X_train)
X_test = token.texts_to_sequences(X_test)

padding

In [10]:
sequence_lengths = [len(seq) for seq in X_train]

# Find the maximum length
max_length = max(sequence_lengths)

In [11]:
max_length

47

In [12]:
MAX_SEQUENCE_LENGTH = 47
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH, padding="post")
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH, padding="post")
X_train.shape, X_test.shape

((7773, 47), (864, 47))

In [19]:
y_train

array([1, 1, 1, ..., 1, 2, 2])

In [14]:
from tensorflow.keras.utils import to_categorical


y_train_labels = np.array(y_train)  # Convert to NumPy array if not already
y_test_labels = np.array(y_test)  # Convert to NumPy array if not already

In [21]:
y_train = np.where(y_train_labels == -1, 0, y_train)
y_train = np.where(y_train_labels == 0, 1, y_train)
y_train = np.where(y_train_labels == 1, 2, y_train)

y_test = np.where(y_test_labels == -1, 0, y_test)
y_test = np.where(y_test_labels == 0, 1, y_test)
y_test = np.where(y_test_labels == 1, 2, y_test)

y_train = to_categorical(y_train, num_classes=3)
y_test = to_categorical(y_test, num_classes=3)


In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout,MaxPooling1D
import tensorflow as tf

vec_size = 300
model = Sequential()
model.add(Embedding(len(token.index_word) + 1, vec_size, input_length=max_length))
model.add(Conv1D(64, 8, activation="relu"))
model.add(MaxPooling1D(2))
model.add(Dropout(0.1))

model.add(Dense(8, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(4, activation='relu'))
model.add(Dropout(0.1))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='linear'))  # Output layer with linear activation


model.compile(loss='mean_squared_error', optimizer=tf.optimizers.Adam(learning_rate=0.0001), metrics=['mae'])
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 47, 300)           3140700   
                                                                 
 conv1d_2 (Conv1D)           (None, 40, 64)            153664    
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 20, 64)           0         
 1D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 20, 64)            0         
                                                                 
 dense_6 (Dense)             (None, 20, 8)             520       
                                                                 
 dropout_7 (Dropout)         (None, 20, 8)             0         
                                                      

In [34]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

epochs = 100
batch_size = 4


early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min')

# Train the model with callbacks
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=32, callbacks=[early_stopping, model_checkpoint])


Epoch 1/100
243/243 [==============================] - 10s 42ms/step - loss: 0.3044 - mae: 0.3489 - val_loss: 0.2980 - val_mae: 0.3527
Epoch 2/100
243/243 [==============================] - 10s 43ms/step - loss: 0.2922 - mae: 0.3563 - val_loss: 0.2865 - val_mae: 0.3599
Epoch 3/100
243/243 [==============================] - 11s 47ms/step - loss: 0.2814 - mae: 0.3634 - val_loss: 0.2763 - val_mae: 0.3669
Epoch 4/100
243/243 [==============================] - 11s 44ms/step - loss: 0.2718 - mae: 0.3703 - val_loss: 0.2673 - val_mae: 0.3736
Epoch 5/100
243/243 [==============================] - 11s 47ms/step - loss: 0.2633 - mae: 0.3769 - val_loss: 0.2595 - val_mae: 0.3801
Epoch 6/100
243/243 [==============================] - 10s 43ms/step - loss: 0.2560 - mae: 0.3832 - val_loss: 0.2526 - val_mae: 0.3864
Epoch 7/100
243/243 [==============================] - 10s 42ms/step - loss: 0.2496 - mae: 0.3894 - val_loss: 0.2467 - val_mae: 0.3923
Epoch 8/100
243/243 [==============================] - 

In [35]:
predictions = model.predict(X_test)

# Convert probabilities to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Convert one-hot encoded y_test back to class labels
true_classes = np.argmax(y_test, axis=1)

# Create a DataFrame to compare predictions with actual values
results_df = pd.DataFrame({
    'Text': [" ".join([token.index_word.get(idx, "") for idx in x if idx != 0]) for x in X_test],
    'Actual Class': true_classes,
    'Predicted Class': predicted_classes,
    'Predicted Probability': np.max(predictions, axis=1)
})

# Show the DataFrame
print(results_df.loc[results_df["Actual Class"]==2])

27/27 [==============================] - 0s 6ms/step
                                                  Text  Actual Class  \
2      department store sales improved 14 eur 10706 mn             2   
10   stable outlook reflects nokia strong market po...             2   
20     cost savings rise 20 mln eur year 2007 oko said             2   
23   planned facility estimated cost around 814 mil...             2   
27   sale healthcare trade business supports oriola...             2   
..                                                 ...           ...   
844  kesko agro lietuva agricultural machinery grai...             2   
856  finnish food industry companies hk ruokatalo a...             2   
860  company makes garden tools scissors consumer g...             2   
861  loans used finance strategic investments shopp...             2   
863  dirk jones head financial institutions client ...             2   

     Predicted Class  Predicted Probability  
2                  0               0

In [ ]:
def predictions(x):
    prediction_probs = model.predict(x)
    predictions = [1 if prob > 0.5 else 0 for prob in prediction_probs]
    return predictions

In [ ]:
test = pd.DataFrame(X_test).iloc[0]

In [ ]:
predictions = model.predict(X_test)

27/27 [==============================] - 0s 5ms/step


In [ ]:
predictions = predictions.flatten() 

In [ ]:
reverse_word_index = {v: k for k, v in token.word_index.items()}

def sequences_to_texts(sequences):
    texts = []
    for sequence in sequences:
        texts.append(' '.join([reverse_word_index.get(i, '?') for i in sequence]))
    return texts

test_phrases = sequences_to_texts(X_test)

In [ ]:
results_df = pd.DataFrame({
    'Phrase': test_phrases,
    'Actual Sentiment': y_test.flatten(),
    'Predicted Probability': predictions
})

print(results_df.head(10))

ValueError: All arrays must be of the same length

In [ ]:
new_headlines = [
    "Stock markets rally as economy shows signs of recovery",
    "Severe weather warnings issued across the country",
    "Tech companies report record earnings this quarter"
]

# Tokenize and pad the new headlines
sequences = token.texts_to_sequences(new_headlines)
padded_sequences = pad_sequences(sequences, maxlen=30, padding='post', truncating='post')

# Make predictions
new_predictions = model.predict(padded_sequences)
new_predictions = new_predictions.flatten()  # Flatten if needed

# Convert probabilities to class labels if needed (for visualization)
predicted_classes = (new_predictions > 0.5).astype("int32")
predicted_classes = np.where(predicted_classes == 0, -1, 1)

# Create DataFrame
new_results_df = pd.DataFrame({
    'Headline': new_headlines,
    'Predicted Probability': new_predictions,
    'Predicted Sentiment': predicted_classes
})

# Show the DataFrame
print(new_results_df)

1/1 [==============================] - 0s 19ms/step
                                            Headline  Predicted Probability  \
0  Stock markets rally as economy shows signs of ...               0.998472   
1  Severe weather warnings issued across the country               0.171261   
2  Tech companies report record earnings this qua...               0.989551   

   Predicted Sentiment  
0                    1  
1                   -1  
2                    1  
